<a href="https://colab.research.google.com/github/sue-bin/SCDC2020/blob/master/eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [83]:
import pandas as pd
import numpy as np
import seaborn as sns
from patsy import dmatrices
import statsmodels.api as sm;
from statsmodels.stats.outliers_influence import variance_inflation_factor
%matplotlib inline

In [64]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [65]:
info_df_raw = pd.read_csv("/content/drive/My Drive/데이터분석/samsung/info1.csv")
feat_df_raw = pd.read_csv("/content/drive/My Drive/데이터분석/samsung/samp_cst_feat.csv")
train_df_raw = pd.read_csv("/content/drive/My Drive/데이터분석/samsung/samp_train.csv")

In [66]:
dtype_df_raw = pd.read_excel("/content/drive/My Drive/데이터분석/samsung/variable_dtype.xlsx")

In [67]:
info_df = info_df_raw.copy()
feat_df = feat_df_raw.copy()
train_df = train_df_raw.copy()

In [68]:
info_df_raw #가맹점은 다 다른데 카테고리가 겹치는 애들은 더 가중치 줄 필요 있을듯

,MRC_ID_DI,Category
0,0,미이용
1,1,홈쇼핑
2,2,종합물
3,3,종합몰
4,4,전문몰
5,5,종합몰
6,6,홈쇼핑
7,7,오픈마켓_소셜
8,8,홈쇼핑
9,9,전문몰


In [69]:
feat_df_raw.head()

,cst_id_di,VAR002,VAR003,VAR004,VAR005,VAR006,VAR007,VAR008,VAR009,VAR010,VAR011,VAR012,VAR013,VAR014,VAR015,VAR016,VAR017,VAR018,VAR019,VAR020,VAR021,VAR022,VAR023,VAR024,VAR025,VAR026,VAR027,VAR028,VAR029,VAR030,VAR031,VAR032,VAR033,VAR034,VAR035,VAR036,VAR037,VAR038,VAR039,VAR040,VAR041,VAR042,VAR043,VAR044,VAR045,VAR046,VAR047,VAR048,VAR049,VAR050,VAR051,VAR052,VAR053,VAR054,VAR055,VAR056,VAR057,VAR058,VAR059,VAR060,VAR061,VAR062,VAR063,VAR064,VAR065,VAR066,VAR067,VAR068,VAR069,VAR070,VAR071,VAR072,VAR073,VAR074,VAR075,VAR076,VAR077,VAR078,VAR079,VAR080,VAR081,VAR082,VAR083,VAR084,VAR085,VAR086,VAR087,VAR088,VAR089,VAR090,VAR091,VAR092,VAR093,VAR094,VAR095,VAR096,VAR097,VAR098,VAR099,VAR100,VAR101,VAR102,VAR103,VAR104,VAR105,VAR106,VAR107,VAR108,VAR109,VAR110,VAR111,VAR112,VAR113,VAR114,VAR115,VAR116,VAR117,VAR118,VAR119,VAR120,VAR121,VAR122,VAR123,VAR124,VAR125,VAR126,VAR127,VAR128,VAR129,VAR130,VAR131,VAR132,VAR133,VAR134,VAR135,VAR136,VAR137,VAR138,VAR139,VAR140,VAR141,VAR142,VAR143,VAR144,VAR145,VAR146,VAR147,VAR148,VAR149,VAR150,VAR151,VAR152,VAR153,VAR154,VAR155,VAR156,VAR157,VAR158,VAR159,VAR160,VAR161,VAR162,VAR163,VAR164,VAR165,VAR166,VAR167,VAR168,VAR169,VAR170,VAR171,VAR172,VAR173,VAR174,VAR175,VAR176,VAR177,var178,VAR179,VAR180,VAR181,VAR182,VAR183,VAR184,VAR185,VAR186,VAR187,VAR188,VAR189,VAR190,VAR191,VAR192,VAR193,VAR194,VAR195,VAR196,VAR197,VAR198,VAR199,VAR200,VAR201,VAR202,VAR203,VAR204,VAR205,VAR206,VAR207,VAR208,VAR209,VAR210,VAR211,VAR212,VAR213,VAR214,VAR215,VAR216,VAR217,VAR218,VAR219,VAR220,VAR221,VAR222,VAR223,VAR224,VAR225,VAR226,VAR227
0,90000000089,-0.06610,0.5280,-0.13607,0.10945,0.06557,0,0.7702,-0.18965,0.1981,0.24149,0.32429,-0.18611,0.06960,1,0.0049,0.8085,0,-0.11899,0.03969,-0.16651,0.12037,-0.01715,-0.09941,-0.28878,0,-0.07481,-0.12742,0.14367,0.06725,0.7702,0.04138,-0.01619,-0.32429,0.11580,0.7179,0.16920,-0.08048,0.28126,-0.11289,-0.02092,0.5381,0.22047,0.17756,0.03794,-0.04432,-0.28743,-0.01164,0.26300,-0.15466,-0.08374,-0.12935,-0.24726,-0.32134,0.39311,-0.32813,0.35632,0.12739,1,-0.18338,-0.07717,0.4688,0.18707,-0.03818,-0.04869,1,1,-0.14361,0.20281,0,0,0.02536,-0.20705,0.03747,0.28517,-0.18100,0,0,-0.20069,0.23156,-0.18709,0.20059,0.04329,0.0025,0.13105,-0.00891,0.23236,0.10216,0.28202,0.0077,-0.09886,0.29902,-0.22713,0,-0.23064,0,0,0,0.06628,0.11613,0.0093,0.03583,0.3578,0.20585,1.0000,-0.29631,0,0.0008,0.26873,0.05674,0,0.02082,0.26616,-0.25601,0.16075,0.15089,0.7286,0.07524,-0.11761,-0.13649,0.16103,0.38476,0.18881,0,0.00524,0.29896,1,0.36885,0.05585,0.06960,0.0068,0.05439,-0.04895,0.22997,0.06607,-0.02500,-0.05500,0.0020,0.07158,-0.11915,-0.09522,0.11580,1,0,0,0.35606,-0.32429,0,0.19451,-0.22885,0.01926,-0.15068,0.13834,-0.16525,0.09025,0.01253,0.20219,0.31263,-0.17855,-0.20515,-0.18453,-0.21851,0.37549,-0.10744,0,0.13043,0.0013,-0.17485,0.18986,-0.07305,0.0069,-0.30157,-0.16382,0.28385,-0.14273,-0.27069,0,0.3766,0,0.06165,0.0021,-0.03891,0.04840,-0.14790,0.26533,-0.00426,0.14765,-0.13332,0.03077,0.23017,0.11764,0.5740,0.06403,0.06184,0.04242,-0.04322,0.0051,0.04523,0,0.07247,0.09631,-0.22864,0.07262,-0.08837,0.01070,0.06340,0.12424,0,0.20392,0.24685,0.01432,-0.00966,0.09746,-0.17616,-0.18956,-0.13676,-0.13279,-0.32829,0.19113,0.05449,0.09471,0.27091,0.01931,0.02938,0.17105,0.12537,0.22197
1,90000000176,-0.09537,0.1347,-0.13541,0.17331,-0.19657,0,0.0616,-0.23104,0.4940,-0.39476,-0.38366,0.39329,0.17549,0,0.3130,0.6564,0,0.31544,0.03794,-0.30237,-0.34538,0.04368,0.04381,-0.19160,0,-0.09801,-0.04593,0.11228,0.11692,0.5661,0.19938,0.01259,-0.17976,0.20081,0.7454,0.24256,0.01363,-0.22807,0.06728,0.17312,0.5369,0.23387,0.04572,0.05406,-0.18782,-0.23752,-0.10148,-0.16942,0.02893,0.03702,0.04855,0.36531,0.24302,-0.30677,-0.20810,-0.12376,0.15054,0,0.22430,0.10513,0.3797,-0.16796,0.01456,-0.10743,0,1,-0.08246,-0.38202,0,0,0.02066,0.28113,0.05687,-0.11689,0.16837,1,1,-0.08043,0.21618,-0.20256,0.23597,0.06233,0.3285,0.05898,0.00544,-0.29231,0.02483,-0.38122,0.0065,0.39975,0.09775,-0.34015

In [70]:
feat_df_raw.dtypes.head(10) #dtypes 엑셀파일 이용해서 카테고리컬 변수 변환

cst_id_di      int64
VAR002       float64
VAR003       float64
VAR004       float64
VAR005       float64
VAR006       float64
VAR007         int64
VAR008       float64
VAR009       float64
VAR010       float64
dtype: object

In [71]:
train_df_raw.head()

,cst_id_di,MRC_ID_DI
0,90000000089,0
1,90000000176,8
2,90000000210,0
3,90000000212,5
4,90000000213,6


In [72]:
dtype_dict = dtype_df_raw.set_index('Variable_Name')['dType'].to_dict()

In [73]:
#object variables
for w in feat_df_raw.columns:
  if w in dtype_dict.keys():
    if dtype_dict[w] == "categorical":
      feat_df[w] = feat_df_raw[w].astype(str)

In [74]:
obj_col = []
numerical_col = [] 
for w in dtype_dict:
  if dtype_dict[w] == "categorical":
    obj_col.append(w)
  else:
    numerical_col.append(w)

In [77]:
for i in obj_col:
  print(feat_df[i].value_counts())
  print()

0    8633
1    1491
Name: VAR007, dtype: int64

0    5430
1    4694
Name: VAR015, dtype: int64

0    8009
1    2115
Name: VAR018, dtype: int64

0    9926
1     198
Name: VAR026, dtype: int64

0    7606
1    2518
Name: VAR059, dtype: int64

0    7324
1    2800
Name: VAR066, dtype: int64

1    8723
0    1401
Name: VAR067, dtype: int64

0    8166
1    1958
Name: VAR070, dtype: int64

0    9314
1     810
Name: VAR071, dtype: int64

0    8769
1    1355
Name: VAR077, dtype: int64

0    9738
1     386
Name: VAR078, dtype: int64

0    9515
1     609
Name: VAR094, dtype: int64

0    9158
1     966
Name: VAR096, dtype: int64

0    8956
1    1168
Name: VAR097, dtype: int64

0    9306
1     818
Name: VAR098, dtype: int64

0    8434
1    1690
Name: VAR107, dtype: int64

0    7949
1    2175
Name: VAR111, dtype: int64

0    7452
1    2672
Name: VAR124, dtype: int64

0    7249
1    2875
Name: VAR127, dtype: int64

0    9952
1     172
Name: VAR143, dtype: int64

0    9485
1     639
Name: VAR144, dtype:

In [13]:
feat_df.dtypes.head(10)

cst_id_di      int64
VAR002       float64
VAR003       float64
VAR004       float64
VAR005       float64
VAR006       float64
VAR007        object
VAR008       float64
VAR009       float64
VAR010       float64
dtype: object

In [14]:
feat_df.head()

,cst_id_di,VAR002,VAR003,VAR004,VAR005,VAR006,VAR007,VAR008,VAR009,VAR010,VAR011,VAR012,VAR013,VAR014,VAR015,VAR016,VAR017,VAR018,VAR019,VAR020,VAR021,VAR022,VAR023,VAR024,VAR025,VAR026,VAR027,VAR028,VAR029,VAR030,VAR031,VAR032,VAR033,VAR034,VAR035,VAR036,VAR037,VAR038,VAR039,VAR040,VAR041,VAR042,VAR043,VAR044,VAR045,VAR046,VAR047,VAR048,VAR049,VAR050,VAR051,VAR052,VAR053,VAR054,VAR055,VAR056,VAR057,VAR058,VAR059,VAR060,VAR061,VAR062,VAR063,VAR064,VAR065,VAR066,VAR067,VAR068,VAR069,VAR070,VAR071,VAR072,VAR073,VAR074,VAR075,VAR076,VAR077,VAR078,VAR079,VAR080,VAR081,VAR082,VAR083,VAR084,VAR085,VAR086,VAR087,VAR088,VAR089,VAR090,VAR091,VAR092,VAR093,VAR094,VAR095,VAR096,VAR097,VAR098,VAR099,VAR100,VAR101,VAR102,VAR103,VAR104,VAR105,VAR106,VAR107,VAR108,VAR109,VAR110,VAR111,VAR112,VAR113,VAR114,VAR115,VAR116,VAR117,VAR118,VAR119,VAR120,VAR121,VAR122,VAR123,VAR124,VAR125,VAR126,VAR127,VAR128,VAR129,VAR130,VAR131,VAR132,VAR133,VAR134,VAR135,VAR136,VAR137,VAR138,VAR139,VAR140,VAR141,VAR142,VAR143,VAR144,VAR145,VAR146,VAR147,VAR148,VAR149,VAR150,VAR151,VAR152,VAR153,VAR154,VAR155,VAR156,VAR157,VAR158,VAR159,VAR160,VAR161,VAR162,VAR163,VAR164,VAR165,VAR166,VAR167,VAR168,VAR169,VAR170,VAR171,VAR172,VAR173,VAR174,VAR175,VAR176,VAR177,var178,VAR179,VAR180,VAR181,VAR182,VAR183,VAR184,VAR185,VAR186,VAR187,VAR188,VAR189,VAR190,VAR191,VAR192,VAR193,VAR194,VAR195,VAR196,VAR197,VAR198,VAR199,VAR200,VAR201,VAR202,VAR203,VAR204,VAR205,VAR206,VAR207,VAR208,VAR209,VAR210,VAR211,VAR212,VAR213,VAR214,VAR215,VAR216,VAR217,VAR218,VAR219,VAR220,VAR221,VAR222,VAR223,VAR224,VAR225,VAR226,VAR227
0,90000000089,-0.06610,0.5280,-0.13607,0.10945,0.06557,0,0.7702,-0.18965,0.1981,0.24149,0.32429,-0.18611,0.06960,1,0.0049,0.8085,0,-0.11899,0.03969,-0.16651,0.12037,-0.01715,-0.09941,-0.28878,0,-0.07481,-0.12742,0.14367,0.06725,0.7702,0.04138,-0.01619,-0.32429,0.11580,0.7179,0.16920,-0.08048,0.28126,-0.11289,-0.02092,0.5381,0.22047,0.17756,0.03794,-0.04432,-0.28743,-0.01164,0.26300,-0.15466,-0.08374,-0.12935,-0.24726,-0.32134,0.39311,-0.32813,0.35632,0.12739,1,-0.18338,-0.07717,0.4688,0.18707,-0.03818,-0.04869,1,1,-0.14361,0.20281,0,0,0.02536,-0.20705,0.03747,0.28517,-0.18100,0,0,-0.20069,0.23156,-0.18709,0.20059,0.04329,0.0025,0.13105,-0.00891,0.23236,0.10216,0.28202,0.0077,-0.09886,0.29902,-0.22713,0,-0.23064,0,0,0,0.06628,0.11613,0.0093,0.03583,0.3578,0.20585,1.0000,-0.29631,0,0.0008,0.26873,0.05674,0,0.02082,0.26616,-0.25601,0.16075,0.15089,0.7286,0.07524,-0.11761,-0.13649,0.16103,0.38476,0.18881,0,0.00524,0.29896,1,0.36885,0.05585,0.06960,0.0068,0.05439,-0.04895,0.22997,0.06607,-0.02500,-0.05500,0.0020,0.07158,-0.11915,-0.09522,0.11580,1,0,0,0.35606,-0.32429,0,0.19451,-0.22885,0.01926,-0.15068,0.13834,-0.16525,0.09025,0.01253,0.20219,0.31263,-0.17855,-0.20515,-0.18453,-0.21851,0.37549,-0.10744,0,0.13043,0.0013,-0.17485,0.18986,-0.07305,0.0069,-0.30157,-0.16382,0.28385,-0.14273,-0.27069,0,0.3766,0,0.06165,0.0021,-0.03891,0.04840,-0.14790,0.26533,-0.00426,0.14765,-0.13332,0.03077,0.23017,0.11764,0.5740,0.06403,0.06184,0.04242,-0.04322,0.0051,0.04523,0,0.07247,0.09631,-0.22864,0.07262,-0.08837,0.01070,0.06340,0.12424,0,0.20392,0.24685,0.01432,-0.00966,0.09746,-0.17616,-0.18956,-0.13676,-0.13279,-0.32829,0.19113,0.05449,0.09471,0.27091,0.01931,0.02938,0.17105,0.12537,0.22197
1,90000000176,-0.09537,0.1347,-0.13541,0.17331,-0.19657,0,0.0616,-0.23104,0.4940,-0.39476,-0.38366,0.39329,0.17549,0,0.3130,0.6564,0,0.31544,0.03794,-0.30237,-0.34538,0.04368,0.04381,-0.19160,0,-0.09801,-0.04593,0.11228,0.11692,0.5661,0.19938,0.01259,-0.17976,0.20081,0.7454,0.24256,0.01363,-0.22807,0.06728,0.17312,0.5369,0.23387,0.04572,0.05406,-0.18782,-0.23752,-0.10148,-0.16942,0.02893,0.03702,0.04855,0.36531,0.24302,-0.30677,-0.20810,-0.12376,0.15054,0,0.22430,0.10513,0.3797,-0.16796,0.01456,-0.10743,0,1,-0.08246,-0.38202,0,0,0.02066,0.28113,0.05687,-0.11689,0.16837,1,1,-0.08043,0.21618,-0.20256,0.23597,0.06233,0.3285,0.05898,0.00544,-0.29231,0.02483,-0.38122,0.0065,0.39975,0.09775,-0.34015

In [15]:
train_df.head(10)

,cst_id_di,MRC_ID_DI
0,90000000089,0
1,90000000176,8
2,90000000210,0
3,90000000212,5
4,90000000213,6
5,90000000263,0
6,90000000322,0
7,90000000354,0
8,90000000407,0
9,90000000408,0


In [16]:
train_df.groupby("MRC_ID_DI").sum()

,cst_id_di
MRC_ID_DI,
0,738001873446410
1,7650019500654
2,1260003033581
3,2880007977965
4,990003299072
5,12870030883335
6,49860128414992
7,59490150776353
8,30330077164853


In [17]:
combine_df = train_df.merge(right= feat_df,how="inner")

In [18]:
combine_df.head(10)

,cst_id_di,MRC_ID_DI,VAR002,VAR003,VAR004,VAR005,VAR006,VAR007,VAR008,VAR009,VAR010,VAR011,VAR012,VAR013,VAR014,VAR015,VAR016,VAR017,VAR018,VAR019,VAR020,VAR021,VAR022,VAR023,VAR024,VAR025,VAR026,VAR027,VAR028,VAR029,VAR030,VAR031,VAR032,VAR033,VAR034,VAR035,VAR036,VAR037,VAR038,VAR039,VAR040,VAR041,VAR042,VAR043,VAR044,VAR045,VAR046,VAR047,VAR048,VAR049,VAR050,VAR051,VAR052,VAR053,VAR054,VAR055,VAR056,VAR057,VAR058,VAR059,VAR060,VAR061,VAR062,VAR063,VAR064,VAR065,VAR066,VAR067,VAR068,VAR069,VAR070,VAR071,VAR072,VAR073,VAR074,VAR075,VAR076,VAR077,VAR078,VAR079,VAR080,VAR081,VAR082,VAR083,VAR084,VAR085,VAR086,VAR087,VAR088,VAR089,VAR090,VAR091,VAR092,VAR093,VAR094,VAR095,VAR096,VAR097,VAR098,VAR099,VAR100,VAR101,VAR102,VAR103,VAR104,VAR105,VAR106,VAR107,VAR108,VAR109,VAR110,VAR111,VAR112,VAR113,VAR114,VAR115,VAR116,VAR117,VAR118,VAR119,VAR120,VAR121,VAR122,VAR123,VAR124,VAR125,VAR126,VAR127,VAR128,VAR129,VAR130,VAR131,VAR132,VAR133,VAR134,VAR135,VAR136,VAR137,VAR138,VAR139,VAR140,VAR141,VAR142,VAR143,VAR144,VAR145,VAR146,VAR147,VAR148,VAR149,VAR150,VAR151,VAR152,VAR153,VAR154,VAR155,VAR156,VAR157,VAR158,VAR159,VAR160,VAR161,VAR162,VAR163,VAR164,VAR165,VAR166,VAR167,VAR168,VAR169,VAR170,VAR171,VAR172,VAR173,VAR174,VAR175,VAR176,VAR177,var178,VAR179,VAR180,VAR181,VAR182,VAR183,VAR184,VAR185,VAR186,VAR187,VAR188,VAR189,VAR190,VAR191,VAR192,VAR193,VAR194,VAR195,VAR196,VAR197,VAR198,VAR199,VAR200,VAR201,VAR202,VAR203,VAR204,VAR205,VAR206,VAR207,VAR208,VAR209,VAR210,VAR211,VAR212,VAR213,VAR214,VAR215,VAR216,VAR217,VAR218,VAR219,VAR220,VAR221,VAR222,VAR223,VAR224,VAR225,VAR226,VAR227
0,90000000089,0,-0.06610,0.5280,-0.13607,0.10945,0.06557,0,0.7702,-0.18965,0.1981,0.24149,0.32429,-0.18611,0.06960,1,0.0049,0.8085,0,-0.11899,0.03969,-0.16651,0.12037,-0.01715,-0.09941,-0.28878,0,-0.07481,-0.12742,0.14367,0.06725,0.7702,0.04138,-0.01619,-0.32429,0.11580,0.7179,0.16920,-0.08048,0.28126,-0.11289,-0.02092,0.5381,0.22047,0.17756,0.03794,-0.04432,-0.28743,-0.01164,0.26300,-0.15466,-0.08374,-0.12935,-0.24726,-0.32134,0.39311,-0.32813,0.35632,0.12739,1,-0.18338,-0.07717,0.4688,0.18707,-0.03818,-0.04869,1,1,-0.14361,0.20281,0,0,0.02536,-0.20705,0.03747,0.28517,-0.18100,0,0,-0.20069,0.23156,-0.18709,0.20059,0.04329,0.0025,0.13105,-0.00891,0.23236,0.10216,0.28202,0.0077,-0.09886,0.29902,-0.22713,0,-0.23064,0,0,0,0.06628,0.11613,0.0093,0.03583,0.3578,0.20585,1.0000,-0.29631,0,0.0008,0.26873,0.05674,0,0.02082,0.26616,-0.25601,0.16075,0.15089,0.7286,0.07524,-0.11761,-0.13649,0.16103,0.38476,0.18881,0,0.00524,0.29896,1,0.36885,0.05585,0.06960,0.0068,0.05439,-0.04895,0.22997,0.06607,-0.02500,-0.05500,0.0020,0.07158,-0.11915,-0.09522,0.11580,1,0,0,0.35606,-0.32429,0,0.19451,-0.22885,0.01926,-0.15068,0.13834,-0.16525,0.09025,0.01253,0.20219,0.31263,-0.17855,-0.20515,-0.18453,-0.21851,0.37549,-0.10744,0,0.13043,0.0013,-0.17485,0.18986,-0.07305,0.0069,-0.30157,-0.16382,0.28385,-0.14273,-0.27069,0,0.3766,0,0.06165,0.0021,-0.03891,0.04840,-0.14790,0.26533,-0.00426,0.14765,-0.13332,0.03077,0.23017,0.11764,0.5740,0.06403,0.06184,0.04242,-0.04322,0.0051,0.04523,0,0.07247,0.09631,-0.22864,0.07262,-0.08837,0.01070,0.06340,0.12424,0,0.20392,0.24685,0.01432,-0.00966,0.09746,-0.17616,-0.18956,-0.13676,-0.13279,-0.32829,0.19113,0.05449,0.09471,0.27091,0.01931,0.02938,0.17105,0.12537,0.22197
1,90000000176,8,-0.09537,0.1347,-0.13541,0.17331,-0.19657,0,0.0616,-0.23104,0.4940,-0.39476,-0.38366,0.39329,0.17549,0,0.3130,0.6564,0,0.31544,0.03794,-0.30237,-0.34538,0.04368,0.04381,-0.19160,0,-0.09801,-0.04593,0.11228,0.11692,0.5661,0.19938,0.01259,-0.17976,0.20081,0.7454,0.24256,0.01363,-0.22807,0.06728,0.17312,0.5369,0.23387,0.04572,0.05406,-0.18782,-0.23752,-0.10148,-0.16942,0.02893,0.03702,0.04855,0.36531,0.24302,-0.30677,-0.20810,-0.12376,0.15054,0,0.22430,0.10513,0.3797,-0.16796,0.01456,-0.10743,0,1,-0.08246,-0.38202,0,0,0.02066,0.28113,0.05687,-0.11689,0.16837,1,1,-0.08043,0.21618,-0.20256,0.23597,0.06233,0.3285,0.05898,0.00544,-0.29231,0.02483,-0.38122,0.0065,0.39975,0.

In [57]:
corr_df = feat_df.corr()

In [59]:
corr_df.values

array([[ 1.        ,  0.01137296,  0.01180641, ...,  0.001397  ,
        -0.00845115, -0.00440571],
       [ 0.01137296,  1.        ,  0.07282754, ..., -0.05603774,
        -0.19744915,  0.24209927],
       [ 0.01180641,  0.07282754,  1.        , ..., -0.45189638,
        -0.51839938,  0.3462779 ],
       ...,
       [ 0.001397  , -0.05603774, -0.45189638, ...,  1.        ,
         0.54312135, -0.22787306],
       [-0.00845115, -0.19744915, -0.51839938, ...,  0.54312135,
         1.        , -0.58786642],
       [-0.00440571,  0.24209927,  0.3462779 , ..., -0.22787306,
        -0.58786642,  1.        ]])

In [ ]:
for i in corr_df.columns:
  

In [60]:
corr_df

,cst_id_di,VAR002,VAR003,VAR004,VAR005,VAR006,VAR008,VAR009,VAR010,VAR011,VAR012,VAR013,VAR014,VAR016,VAR017,VAR019,VAR020,VAR021,VAR022,VAR023,VAR024,VAR025,VAR027,VAR028,VAR029,VAR030,VAR031,VAR032,VAR033,VAR034,VAR035,VAR036,VAR037,VAR038,VAR039,VAR040,VAR041,VAR042,VAR043,VAR044,VAR045,VAR046,VAR047,VAR048,VAR049,VAR050,VAR051,VAR052,VAR053,VAR054,VAR055,VAR056,VAR057,VAR058,VAR060,VAR061,VAR062,VAR063,VAR064,VAR065,VAR068,VAR069,VAR072,VAR073,VAR074,VAR075,VAR076,VAR079,VAR080,VAR081,VAR082,VAR083,VAR084,VAR085,VAR086,VAR087,VAR088,VAR089,VAR090,VAR091,VAR092,VAR093,VAR095,VAR099,VAR100,VAR101,VAR102,VAR103,VAR104,VAR105,VAR106,VAR108,VAR109,VAR110,VAR112,VAR113,VAR114,VAR115,VAR116,VAR117,VAR118,VAR119,VAR120,VAR121,VAR122,VAR123,VAR125,VAR126,VAR128,VAR129,VAR130,VAR131,VAR132,VAR133,VAR134,VAR135,VAR136,VAR137,VAR138,VAR139,VAR140,VAR141,VAR142,VAR146,VAR147,VAR149,VAR150,VAR151,VAR152,VAR153,VAR154,VAR155,VAR156,VAR157,VAR158,VAR159,VAR160,VAR161,VAR162,VAR163,VAR164,VAR166,VAR167,VAR168,VAR169,VAR170,VAR171,VAR172,VAR173,VAR174,VAR175,VAR176,var178,VAR180,VAR181,VAR182,VAR183,VAR184,VAR185,VAR186,VAR187,VAR188,VAR189,VAR190,VAR191,VAR192,VAR193,VAR194,VAR195,VAR196,VAR197,VAR198,VAR200,VAR201,VAR202,VAR203,VAR204,VAR205,VAR206,VAR207,VAR209,VAR210,VAR211,VAR212,VAR213,VAR214,VAR215,VAR216,VAR217,VAR218,VAR219,VAR220,VAR221,VAR222,VAR223,VAR224,VAR225,VAR226,VAR227
cst_id_di,1.000000,0.011373,0.011806,-0.000047,-0.016298,0.010984,0.002119,0.009057,-0.002224,-0.002349,0.002004,-0.004851,-0.015961,-0.010277,-0.017196,-0.002964,0.005066,0.009025,-0.014293,0.005525,0.029339,0.009360,0.019961,0.006319,-0.020076,-0.008860,-0.009483,-0.007188,0.021101,0.015482,-0.001620,0.019588,-0.018970,0.005558,-0.007186,-0.015073,0.008619,-0.014110,-0.009369,-0.000569,-0.009982,0.007837,0.014411,0.019523,-0.005619,0.011231,0.004946,-0.015059,0.014991,-0.001632,-0.011295,0.010637,-0.020396,-0.015655,-0.002295,-0.006994,0.002046,-0.021432,-0.018059,0.012557,0.019002,-0.000814,-0.024888,0.005740,-0.002479,0.004206,0.002524,0.002489,-0.012655,0.014784,-0.018672,-0.010816,0.000033,-0.005761,-0.012377,0.004507,-0.002807,-0.001964,-0.008055,-0.001104,-0.015770,0.007830,0.002510,-0.008201,-0.011848,-0.014134,-0.006818,-0.006064,-0.011698,0.007200,0.008270,0.001931,-0.005816,-0.010173,-0.007089,0.005141,-0.011072,-0.009355,0.005831,-0.011310,0.007750,-0.007717,-0.001078,-0.015734,-0.011975,0.002682,-0.007564,-0.010287,-0.006298,-0.016672,0.001569,-0.011563,-0.015961,0.012064,0.001256,-0.005733,0.021941,-0.010616,0.001304,0.002863,0.007250,0.008625,-0.000022,-0.006159,0.009063,-0.007723,-0.010137,-0.005081,0.021805,-0.008735,0.008868,-0.007014,-0.029438,-0.012953,-0.017408,0.023422,-0.013348,0.017405,0.019523,0.001361,0.014594,-0.006651,0.009958,-0.001731,-0.007763,0.032315,0.005213,0.016738,0.019047,-0.009452,0.019674,-0.007398,-0.009188,-0.010826,-0.011664,0.005967,-0.004370,0.005903,-0.008418,-0.013769,-0.005283,0.024820,-0.001205,-0.014603,-0.012321,0.005420,-0.005369,-0.008619,-0.010483,-0.005458,0.007783,0.002612,0.004974,-0.026346,-0.004711,0.003908,0.012364,0.019010,-0.008103,-0.015888,-0.006828,-0.013583,0.006608,0.002803,0.002071,0.012115,0.004891,0.000292,0.020840,0.011470,-0.006953,-0.003614,-0.008939,-0.010470,-0.004318,-0.006528,0.001397,-0.008451,-0.004406
VAR002,0.011373,1.000000,0.072828,0.042407,-0.088899,0.229318,-0.046654,0.069127,-0.101209,0.227637,0.273233,-0.199249,-0.181866,0.017604,-0.129183,-0.138205,0.055740,0.210896,0.154242,-0.086632,0.111107,0.113215,0.192300,-0.030270,-0.089762,-0.073261,-0.009721,-0.141696,-0.192211,0.139570,-0.167327,-0.030738,-0.115261,0.018525,0.145358,-0.079530,-0.153539,-0.145533,-0.106780,-0.058567,-0.019092,0.262674,0.154161,0.071033,0.139003,0.059948,-0.147889,-0.156510,-0.203645,-0.167736,0.224391,0.133262,0.059188,-0.150550,-0.267390,-0.193944,0.029529,-0.048729,0.173563,0.380837,0.103095,0.225785,0.363241,-0.227110,-0.044168,0.096929,-0.251478,-0.197248,-0.149761,0.106263,-0.039795,-0.043

In [78]:
var_list = combine_df.columns[2:]

In [79]:
var_list

Index(['VAR002', 'VAR003', 'VAR004', 'VAR005', 'VAR006', 'VAR007', 'VAR008',
       'VAR009', 'VAR010', 'VAR011',
       ...
       'VAR218', 'VAR219', 'VAR220', 'VAR221', 'VAR222', 'VAR223', 'VAR224',
       'VAR225', 'VAR226', 'VAR227'],
      dtype='object', length=226)

In [80]:
var_str = "+".join(var_list)

In [81]:
var_str

'VAR002+VAR003+VAR004+VAR005+VAR006+VAR007+VAR008+VAR009+VAR010+VAR011+VAR012+VAR013+VAR014+VAR015+VAR016+VAR017+VAR018+VAR019+VAR020+VAR021+VAR022+VAR023+VAR024+VAR025+VAR026+VAR027+VAR028+VAR029+VAR030+VAR031+VAR032+VAR033+VAR034+VAR035+VAR036+VAR037+VAR038+VAR039+VAR040+VAR041+VAR042+VAR043+VAR044+VAR045+VAR046+VAR047+VAR048+VAR049+VAR050+VAR051+VAR052+VAR053+VAR054+VAR055+VAR056+VAR057+VAR058+VAR059+VAR060+VAR061+VAR062+VAR063+VAR064+VAR065+VAR066+VAR067+VAR068+VAR069+VAR070+VAR071+VAR072+VAR073+VAR074+VAR075+VAR076+VAR077+VAR078+VAR079+VAR080+VAR081+VAR082+VAR083+VAR084+VAR085+VAR086+VAR087+VAR088+VAR089+VAR090+VAR091+VAR092+VAR093+VAR094+VAR095+VAR096+VAR097+VAR098+VAR099+VAR100+VAR101+VAR102+VAR103+VAR104+VAR105+VAR106+VAR107+VAR108+VAR109+VAR110+VAR111+VAR112+VAR113+VAR114+VAR115+VAR116+VAR117+VAR118+VAR119+VAR120+VAR121+VAR122+VAR123+VAR124+VAR125+VAR126+VAR127+VAR128+VAR129+VAR130+VAR131+VAR132+VAR133+VAR134+VAR135+VAR136+VAR137+VAR138+VAR139+VAR140+VAR141+VAR142+VAR143+VAR14

In [ ]:
y, X = dmatrices('MRC_ID_DI ~ VAR002+VAR003+VAR004+VAR005+VAR006+VAR007+VAR008+VAR009+VAR010+VAR011+VAR012+VAR013+VAR014+VAR015+VAR016+VAR017+VAR018+VAR019+VAR020+VAR021+VAR022+VAR023+VAR024+VAR025+VAR026+VAR027+VAR028+VAR029+VAR030+VAR031+VAR032+VAR033+VAR034+VAR035+VAR036+VAR037+VAR038+VAR039+VAR040+VAR041+VAR042+VAR043+VAR044+VAR045+VAR046+VAR047+VAR048+VAR049+VAR050+VAR051+VAR052+VAR053+VAR054+VAR055+VAR056+VAR057+VAR058+VAR059+VAR060+VAR061+VAR062+VAR063+VAR064+VAR065+VAR066+VAR067+VAR068+VAR069+VAR070+VAR071+VAR072+VAR073+VAR074+VAR075+VAR076+VAR077+VAR078+VAR079+VAR080+VAR081+VAR082+VAR083+VAR084+VAR085+VAR086+VAR087+VAR088+VAR089+VAR090+VAR091+VAR092+VAR093+VAR094+VAR095+VAR096+VAR097+VAR098+VAR099+VAR100+VAR101+VAR102+VAR103+VAR104+VAR105+VAR106+VAR107+VAR108+VAR109+VAR110+VAR111+VAR112+VAR113+VAR114+VAR115+VAR116+VAR117+VAR118+VAR119+VAR120+VAR121+VAR122+VAR123+VAR124+VAR125+VAR126+VAR127+VAR128+VAR129+VAR130+VAR131+VAR132+VAR133+VAR134+VAR135+VAR136+VAR137+VAR138+VAR139+VAR140+VAR141+VAR142+VAR143+VAR144+VAR145+VAR146+VAR147+VAR148+VAR149+VAR150+VAR151+VAR152+VAR153+VAR154+VAR155+VAR156+VAR157+VAR158+VAR159+VAR160+VAR161+VAR162+VAR163+VAR164+VAR165+VAR166+VAR167+VAR168+VAR169+VAR170+VAR171+VAR172+VAR173+VAR174+VAR175+VAR176+VAR177+var178+VAR179+VAR180+VAR181+VAR182+VAR183+VAR184+VAR185+VAR186+VAR187+VAR188+VAR189+VAR190+VAR191+VAR192+VAR193+VAR194+VAR195+VAR196+VAR197+VAR198+VAR199+VAR200+VAR201+VAR202+VAR203+VAR204+VAR205+VAR206+VAR207+VAR208+VAR209+VAR210+VAR211+VAR212+VAR213+VAR214+VAR215+VAR216+VAR217+VAR218+VAR219+VAR220+VAR221+VAR222+VAR223+VAR224+VAR225+VAR226+VAR227', combine_df, return_type = 'dataframe')
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns 

In [86]:
vif

,VIF Factor,features
0,0.000000e+00,Intercept
1,inf,VAR007[T.1]
2,1.858853e+00,VAR015[T.1]
3,inf,VAR018[T.1]
4,1.334723e+00,VAR026[T.1]
5,inf,VAR059[T.1]
6,1.647344e+00,VAR066[T.1]
7,1.407752e+00,VAR067[T.1]
8,1.672975e+00,VAR070[T.1]
9,3.985155e+02,VAR071[T.1]


In [87]:
import math

In [93]:
vif =vif.sort_values("VIF Factor",ascending=False)

In [94]:
vif["VIF Factor"] = vif["VIF Factor"].round()

In [95]:
vif

,VIF Factor,features
17,inf,VAR111[T.1]
16,inf,VAR107[T.1]
3,inf,VAR018[T.1]
5,inf,VAR059[T.1]
27,inf,VAR199[T.1]
26,inf,VAR179[T.1]
19,inf,VAR127[T.1]
18,inf,VAR124[T.1]
14,inf,VAR097[T.1]
1,inf,VAR007[T.1]
